# **Building a Convolutional Neural Network with Keras**

In [ ]:
#Tensorflow version

import tensorflow as tf
print(tf.__version__)

2.9.0


In [ ]:
#Importing libraries

import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Conv2D, Dense, Flatten, BatchNormalization, Dropout, MaxPooling2D, GlobalMaxPooling2D
from keras.models import Model
import matplotlib.pyplot as plt
from keras.applications.efficientnet import EfficientNetB0
from keras.preprocessing import image
#from keras.preprocessing.image import load_img
from keras.models import Sequential
import numpy as np
from tensorflow.keras import regularizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255)
validation_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
train_dir = os.path.join('directory/training/')
validation_dir = os.path.join('directory/validation/')

In [ ]:
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(96, 96),
    batch_size=32,
    class_mode='binary',
    )

val_data = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(96,96),
    batch_size=32,
    class_mode='binary',
    )

Found 16429 images belonging to 2 classes.
Found 3270 images belonging to 2 classes.


In [ ]:
model = tf.keras.models.Sequential([

    # First convolution layer
    tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=(96, 96, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    # Second convolution layer
    tf.keras.layers.Conv1D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    # Third convolution layer
    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv1D(128, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),


    # Flatten the pooled feature maps
    tf.keras.layers.Flatten(),

    tf.keras.layers.Dropout(0.5),

    # Fully connected hidden layer
    tf.keras.layers.Dense(96, activation='relu'),

    # Output layer
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
#print model summary

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, 96, 94, 32)        320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 48, 47, 32)       0         
 2D)                                                             
                                                                 
 conv1d_5 (Conv1D)           (None, 48, 45, 64)        6208      
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 24, 22, 64)       0         
 2D)                                                             
                                                                 
 conv1d_6 (Conv1D)           (None, 24, 20, 128)       24704     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 12, 10, 128)     

In [ ]:
#Performance evaluation Metrics delcaration

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import SensitivityAtSpecificity,SpecificityAtSensitivity,Recall,Precision, AUC

model.compile(loss='binary_crossentropy',
              optimizer=Adam(learning_rate=0.0001),
              metrics=['accuracy',SensitivityAtSpecificity(0.5),SpecificityAtSensitivity(0.5),Recall(0.5),Precision(0.5), AUC(num_thresholds=200,curve='ROC')])


In [ ]:
class mycallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy')>0.99):
            print("\n Reached 99% accuracy so cancelling training!")
            self.model.stop_training=True

mycallback=mycallback()

In [ ]:
#Model Execution

from keras.callbacks import ModelCheckpoint, EarlyStopping

# Initializing Device Specification
device_spec = tf.DeviceSpec(job ="localhost", replica = 0, device_type = "GPU")

# Printing the DeviceSpec
print('Device Spec: ', device_spec.to_string())

# Enabling device logging
tf.debugging.set_log_device_placement(True)

# Specifying the device
with tf.device(device_spec):

#with tf.device('/gpu:10'):

    checkpoint = ModelCheckpoint("ECG_Model_Lead_5.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
    mycallback=tf.keras.callbacks.EarlyStopping(monitor="val_accuracy", patience=150, mode="auto")

    history = model.fit(
          train_data,
          steps_per_epoch=10,
          epochs=500,
          verbose=1,
          validation_data = val_data,
          validation_steps=5,
          callbacks = [mycallback,checkpoint]
    )

Device Spec:  /job:localhost/replica:0/device:GPU:*
Epoch 1/500
 9/10 [==========================>...] - ETA: 0s - loss: 0.6885 - accuracy: 0.5451 - sensitivity_at_specificity_1: 0.5091 - specificity_at_sensitivity_1: 0.5056 - recall_1: 0.4455 - precision_1: 0.4118 - auc_1: 0.5311
Epoch 1: val_accuracy improved from -inf to 0.54375, saving model to ECG_Model_Lead_5.h5
10/10 [==============================] - 2s 86ms/step - loss: 0.6858 - accuracy: 0.5594 - sensitivity_at_specificity_1: 0.5417 - specificity_at_sensitivity_1: 0.5050 - recall_1: 0.4083 - precision_1: 0.4118 - auc_1: 0.5390 - val_loss: 0.6897 - val_accuracy: 0.5437 - val_sensitivity_at_specificity_1: 0.4384 - val_specificity_at_sensitivity_1: 0.0575 - val_recall_1: 0.0000e+00 - val_precision_1: 0.0000e+00 - val_auc_1: 0.5425
Epoch 2/500
 9/10 [==========================>...] - ETA: 0s - loss: 0.6762 - accuracy: 0.5938 - sensitivity_at_specificity_1: 0.5214 - specificity_at_sensitivity_1: 0.5205 - recall_1: 0.0000e+00 - pre

In [ ]:
Training_Accuracy=history.history['accuracy']
Validation_Accuracy=history.history['val_accuracy']
Validation_Specificity=history.history['val_specificity_at_sensitivity_1']
Validation_Sensitivity=history.history['val_sensitivity_at_specificity_1']
Validation_Recall=history.history['val_recall_1']
Validation_Precision=history.history['val_precision_1']
Validation_Loss=history.history['val_loss']
Validation_AUC=history.history['val_auc_1']

print("Training Accuracy: ",np.max(Training_Accuracy))
print("Validation Accuracy: ",np.max(Validation_Accuracy))
print("Validation Specificity: ",np.max(Validation_Specificity))
print("Validation Sensitivity: ",np.max(Validation_Sensitivity))
print("Validation Recall: ",np.max(Validation_Recall))
print("Validation Precision: ",np.max(Validation_Precision))
print("Validation Loss: ",np.min(Validation_Loss))
print("Validation AUC: ",np.max(Validation_AUC))

Training Accuracy:  0.831250011920929
Validation Accuracy:  0.8374999761581421
Validation Specificity:  1.0
Validation Sensitivity:  1.0
Validation Recall:  0.9285714030265808
Validation Precision:  0.9444444179534912
Validation Loss:  0.41335782408714294
Validation AUC:  0.9159216284751892


In [ ]:
#Proposed Testing Architecture

from keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tensorflow.keras import layers
import cv2
import shutil
import scipy
from scipy.signal import butter, filtfilt
from sklearn.metrics import roc_auc_score

# Normalization layer
nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

def create_image(file_path, save_dir):
    # Load file
    x = np.load(file_path)

    x_size=np.array(x).flatten()
    if len(x_size) <= 6000:
        print(f"Skipping file: {file_path} due to insufficient length.")
        return

    # Resample the signals from 300 Hz to 100 Hz
    original_frequency = 300  # Sampling frequency of original signals (in Hz)
    target_frequency = 100  # Target sampling frequency (in Hz)
    # Calculate the resampling ratio
    resampling_ratio = target_frequency / original_frequency
    # Initialize an empty array to store resampled signals
    resampled_signals = []
    # Resample each signal in the dataset
    for signal_data in x:
        # Perform resampling
        resampled_signal = scipy.signal.resample(signal_data, int(len(signal_data) * resampling_ratio))
        resampled_signals.append(resampled_signal)

    # Convert the list of resampled signals back to a NumPy array
    resampled_signals = np.array(resampled_signals)


    signal = resampled_signals[:]
    signal = np.array(signal).flatten()

    #Checking and Correction of Signal Deflection
    b, a = butter(4, 0.02, btype='high', analog=False)
    filtered_ecg = filtfilt(b, a, signal)
    max_index = np.argmax(np.abs(filtered_ecg[600:len(filtered_ecg)-50]))
    y = filtered_ecg[600:len(filtered_ecg)-50]
    max_amplitude = y[max_index]

    if max_amplitude >= 0:
        signal = signal
    else:
        signal = -(signal)


    # Divide the signal into three parts
    sub_image_length = 1000
    print("Signal Length: ",len(signal))
    for i in range(3):
        start_idx = i * sub_image_length
        end_idx = (i + 1) * sub_image_length

        sub_signal = signal[start_idx:end_idx]
        normalized_array = (sub_signal / max(sub_signal)).clip(-1, 1)

        plt.figure(figsize=(1.25, 1.25))
        plt.plot(normalized_array, 'gray')
        plt.axis("off")

        sub_image_path = os.path.join(save_dir, f"{i + 1}.png")
        plt.savefig(sub_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

#Image Processing and Classification by the Proposed CNN Model
def load_image(img_path, target_size=(96, 96)):
    img = image.load_img(img_path, target_size=target_size)

    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)
    img_tensor = nrmzln_layer(img_tensor)

    return img_tensor

def predict_class(model, img_path, threshold=0.64):

    img_tensor = load_image(img_path)
    pred = model.predict(img_tensor)
    return 1 if pred >= threshold else 0, pred[0][0]

def evaluate_test_folder(model, test_folder):
    class_labels = sorted(os.listdir(test_folder))
    total_samples = 0
    correct_predictions = 0
    misclassifications = {0: 0, 1: 0}
    skipped_files = []
    true_labels = []
    predicted_probabilities = []

    for class_label in class_labels:
        class_path = os.path.join(test_folder, class_label)
        for npy_file in os.listdir(class_path):
            npy_path = os.path.join(class_path, npy_file)

            # Create three sub-images using create_image function
            save_dir = "directory/temp_images/"
            create_image(npy_path, save_dir)

            # Predict class label for each sub-image
            votes = []
            votes_avg = []
            for i in range(1, 4):
                sub_image_path = os.path.join(save_dir, f"{i}.png")
                predicted_class, _ = predict_class(model, sub_image_path)
                print(f"predicted_class {predicted_class}, _ {_}")
                votes.append(predicted_class)
                votes_avg.append(_)

            print("votes for ",sub_image_path," is: ",votes)
            # Determine the overall label based on voting

            # Sum up the confidence values for each class
            sum_confidence_1 = sum([conf for vote, conf in zip(votes, votes_avg) if vote == 1])
            sum_confidence_0 = sum([conf for vote, conf in zip(votes, votes_avg) if vote == 0])
            print(f"sum_confidence_1 {sum_confidence_1}, sum_confidence_0 {sum_confidence_0}")
            # Determine the overall label based on voting
            overall_label = 1 if sum_confidence_1 > sum_confidence_0 else 0
            print("Overall Label:", overall_label)

            # Get true class label based on folder name
            true_class = 0 if class_label == 'disease' else 1

            # Print details for each file
            print(f"File: {npy_file}, Actual Label: {true_class}, Predicted Label: {overall_label}")

            # Update metrics
            total_samples += 1
            correct_predictions += 1 if overall_label == true_class else 0
            true_labels.append(true_class)
            predicted_probabilities.append(overall_label)

            # Track misclassifications
            if overall_label != true_class:
                misclassifications[true_class] += 1

    accuracy = correct_predictions / total_samples
    auc = roc_auc_score(true_labels, predicted_probabilities)

    print("True labels: ",true_labels)
    print("pred labels: ",predicted_probabilities)
    print(f"\nOverall Accuracy: {accuracy * 100:.2f}%")
    print(f"AUC: {auc}")
    print("Misclassifications:")
    for class_label, count in misclassifications.items():
        print(f"Class {class_label}: {count} misclassifications")

    if skipped_files:
        print("\nSkipped Files:")
        for skipped_file in skipped_files:
            print(skipped_file)

    # Clean up temporary images
    files = glob.glob(r'directory/temp_images/*')
    for items in files:
        os.remove(items)

# Load the model
model_path = "directory/ECG_Model_Lead_5.h5"
model = load_model(model_path)

# Test folder path
test_folder_path = 'directory/test_set/'

# Evaluate the test folder
evaluate_test_folder(model, test_folder_path)


Signal Length:  3000
1/1 [==============================] - 0s 126ms/step
predicted_class 0, _ 0.5335848331451416
1/1 [==============================] - 0s 14ms/step
predicted_class 1, _ 0.7760696411132812
1/1 [==============================] - 0s 13ms/step
predicted_class 1, _ 0.7280526161193848
votes for  /home/ubuntu/From_Laptop/12 leads vs 1 lead/temp_images/3.png  is:  [0, 1, 1]
sum_confidence_1 1.504122257232666, sum_confidence_0 0.5335848331451416
Overall Label: 1
File: A00473.npy, Actual Label: 0, Predicted Label: 1
Signal Length:  3000
1/1 [==============================] - 0s 14ms/step
predicted_class 0, _ 0.529435396194458
1/1 [==============================] - 0s 14ms/step
predicted_class 0, _ 0.5923927426338196
1/1 [==============================] - 0s 13ms/step
predicted_class 0, _ 0.2900576591491699
votes for  /home/ubuntu/From_Laptop/12 leads vs 1 lead/temp_images/3.png  is:  [0, 0, 0]
sum_confidence_1 0, sum_confidence_0 1.4118857979774475
Overall Label: 0
File: A00520

In [ ]:
#Proposed Testing Architecture

from keras.models import load_model
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
import os
import glob
from tensorflow.keras import layers
import cv2
import shutil
import scipy
from scipy.signal import butter, filtfilt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Normalization layer
nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

def create_image(file_path, save_dir):
    # Load file
    x = np.load(file_path)

    x_size=np.array(x).flatten()
    if len(x_size) <= 6000:
        print(f"Skipping file: {file_path} due to insufficient length.")
        return

    # Resample the signals from 300 Hz to 100 Hz
    original_frequency = 300  # Sampling frequency of original signals (in Hz)
    target_frequency = 100  # Target sampling frequency (in Hz)
    # Calculate the resampling ratio
    resampling_ratio = target_frequency / original_frequency
    # Initialize an empty array to store resampled signals
    resampled_signals = []
    # Resample each signal in the dataset
    for signal_data in x:
        # Perform resampling
        resampled_signal = scipy.signal.resample(signal_data, int(len(signal_data) * resampling_ratio))
        resampled_signals.append(resampled_signal)

    # Convert the list of resampled signals back to a NumPy array
    resampled_signals = np.array(resampled_signals)


    signal = resampled_signals[:]
    signal = np.array(signal).flatten()

    #Checking and Correction of Signal Deflection
    b, a = butter(4, 0.02, btype='high', analog=False)
    filtered_ecg = filtfilt(b, a, signal)
    max_index = np.argmax(np.abs(filtered_ecg[600:len(filtered_ecg)-50]))
    y = filtered_ecg[600:len(filtered_ecg)-50]
    max_amplitude = y[max_index]

    if max_amplitude >= 0:
        signal = signal
    else:
        signal = -(signal)


    # Divide the signal into three parts
    sub_image_length = 1000
    print("Signal Length: ",len(signal))
    for i in range(3):
        start_idx = i * sub_image_length
        end_idx = (i + 1) * sub_image_length

        sub_signal = signal[start_idx:end_idx]
        normalized_array = (sub_signal / max(sub_signal)).clip(-1, 1)

        plt.figure(figsize=(1.25, 1.25))
        plt.plot(normalized_array, 'gray')
        plt.axis("off")

        sub_image_path = os.path.join(save_dir, f"{i + 1}.png")
        plt.savefig(sub_image_path, bbox_inches='tight', pad_inches=0)
        plt.close()

#Image Processing and Classification by the Proposed CNN Model
def load_image(img_path, target_size=(96, 96)):
    img = image.load_img(img_path, target_size=target_size)

    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)
    nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)
    img_tensor = nrmzln_layer(img_tensor)

    return img_tensor

def predict_class(model, img_path, threshold=0.64):

    img_tensor = load_image(img_path)
    pred = model.predict(img_tensor)
    return 1 if pred >= threshold else 0, pred[0][0]

def evaluate_test_folder(model, test_folder):
    class_labels = sorted(os.listdir(test_folder))
    total_samples = 0
    correct_predictions = 0
    misclassifications = {0: 0, 1: 0}
    skipped_files = []
    true_labels = []
    predicted_probabilities = []

    for class_label in class_labels:
        class_path = os.path.join(test_folder, class_label)
        for npy_file in os.listdir(class_path):
            npy_path = os.path.join(class_path, npy_file)

            # Create three sub-images using create_image function
            save_dir = "directory/temp_images/"
            create_image(npy_path, save_dir)

            # Predict class label for each sub-image
            votes = []
            votes_avg = []
            for i in range(1, 4):
                sub_image_path = os.path.join(save_dir, f"{i}.png")
                predicted_class, _ = predict_class(model, sub_image_path)
                print(f"predicted_class {predicted_class}, _ {_}")
                votes.append(predicted_class)
                votes_avg.append(_)

            print("votes for ",sub_image_path," is: ",votes)
            # Determine the overall label based on voting

            # Sum up the confidence values for each class
            sum_confidence_1 = sum([conf for vote, conf in zip(votes, votes_avg) if vote == 1])
            sum_confidence_0 = sum([conf for vote, conf in zip(votes, votes_avg) if vote == 0])
            print(f"sum_confidence_1 {sum_confidence_1}, sum_confidence_0 {sum_confidence_0}")
            # Determine the overall label based on voting
            overall_label = 1 if sum_confidence_1 > sum_confidence_0 else 0
            print("Overall Label:", overall_label)

            # Get true class label based on folder name
            true_class = 0 if class_label == 'disease' else 1

            # Print details for each file
            print(f"File: {npy_file}, Actual Label: {true_class}, Predicted Label: {overall_label}")

            # Update metrics
            total_samples += 1
            correct_predictions += 1 if overall_label == true_class else 0
            true_labels.append(true_class)
            predicted_probabilities.append(overall_label)

            # Track misclassifications
            if overall_label != true_class:
                misclassifications[true_class] += 1

    accuracy = correct_predictions / total_samples
    auc = roc_auc_score(true_labels, predicted_probabilities)

    print("True labels: ",true_labels)
    print("pred labels: ",predicted_probabilities)
    print(f"\nOverall Accuracy: {accuracy * 100:.2f}%")
    print(f"AUC: {auc}")
    print("Misclassifications:")
    for class_label, count in misclassifications.items():
        print(f"Class {class_label}: {count} misclassifications")

    if skipped_files:
        print("\nSkipped Files:")
        for skipped_file in skipped_files:
            print(skipped_file)

                # Generate confusion matrix
    cm = confusion_matrix(true_labels, predicted_probabilities)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['Disease', 'Normal'])
    disp.plot(cmap='OrRd', values_format='d', ax=plt.gca(), colorbar=False)
    #plt.title("Confusion Matrix", fontsize=16)
    plt.xlabel("Predicted Labels", fontsize=14)
    plt.ylabel("True Labels", fontsize=14)
    plt.xticks(fontsize=13)
    plt.yticks(fontsize=13)

    # Increase font size for the sample counts inside the matrix
    for text in disp.text_.ravel():
        text.set_fontsize(14)

    # Save the confusion matrix plot
    plt.savefig("directory/confusion_matrix_lead_5.png", dpi=800, bbox_inches='tight')

    # Clean up temporary images
    files = glob.glob(r'directory/temp_images/*')
    for items in files:
        os.remove(items)

# Load the model
model_path = "directory/ECG_Model_Lead_5.h5"
model = load_model(model_path)

# Test folder path
test_folder_path = 'directory/test_set/'

# Evaluate the test folder
evaluate_test_folder(model, test_folder_path)


2024-03-22 10:04:44.847773: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-22 10:04:44.867856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-22 10:04:44.868094: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-22 10:04:44.868755: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

Signal Length:  3000


2024-03-22 10:04:46.136365: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100


1/1 [==============================] - 1s 1s/step
predicted_class 0, _ 0.5335848331451416
1/1 [==============================] - 0s 13ms/step
predicted_class 1, _ 0.7760696411132812
1/1 [==============================] - 0s 13ms/step
predicted_class 1, _ 0.7280526161193848
votes for  /home/ubuntu/From_Laptop/12 leads vs 1 lead/temp_images/3.png  is:  [0, 1, 1]
sum_confidence_1 1.504122257232666, sum_confidence_0 0.5335848331451416
Overall Label: 1
File: A00473.npy, Actual Label: 0, Predicted Label: 1
Signal Length:  3000
1/1 [==============================] - 0s 13ms/step
predicted_class 0, _ 0.529435396194458
1/1 [==============================] - 0s 13ms/step
predicted_class 0, _ 0.5923927426338196
1/1 [==============================] - 0s 13ms/step
predicted_class 0, _ 0.2900576591491699
votes for  /home/ubuntu/From_Laptop/12 leads vs 1 lead/temp_images/3.png  is:  [0, 0, 0]
sum_confidence_1 0, sum_confidence_0 1.4118857979774475
Overall Label: 0
File: A00520.npy, Actual Label: 0, P

In [ ]:
#Basic Model Test Dataset Prediction (with only Denoising and Normalization)

from keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from tensorflow.keras import layers
import cv2

# Normalization layer
nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)

def load_image(img_path, target_size=(96, 96)):
    img = image.load_img(img_path, target_size=target_size)

    img_tensor = image.img_to_array(img)
    img_tensor = np.expand_dims(img_tensor, axis=0)

    # Replace normalization with Keras Rescaling layer
    nrmzln_layer = layers.experimental.preprocessing.Rescaling(1./255)
    img_tensor = nrmzln_layer(img_tensor)

    return img_tensor

def predict_class(model, img_path, threshold=0.20):
    img_tensor = load_image(img_path)
    print(img_tensor.shape)
    pred = model.predict(img_tensor)
    return 1 if pred >= threshold else 0, pred[0][0]

def evaluate_test_folder(model, test_folder):
    class_labels = sorted(os.listdir(test_folder))
    total_samples = 0
    correct_predictions = 0
    misclassifications = {0: 0, 1: 0}

    for class_label in class_labels:
        class_path = os.path.join(test_folder, class_label)
        for image_file in os.listdir(class_path):
            image_path = os.path.join(class_path, image_file)

            # Predict class label using the model
            predicted_class, prediction_value = predict_class(model, image_path)

            # Get true class label based on folder name
            true_class = 0 if class_label == 'disease' else 1

            # Print details for each image
            print(f"Image: {image_file}, Actual Label: {true_class}, Predicted Label: {predicted_class}, Prediction Value: {prediction_value}")

            # Update metrics
            total_samples += 1
            correct_predictions += 1 if predicted_class == true_class else 0

            # Track misclassifications
            if predicted_class != true_class:
                misclassifications[true_class] += 1

    accuracy = correct_predictions / total_samples
    print(f"\nOverall Accuracy: {accuracy * 100:.2f}%")
    print("misclassifications")
    for class_label, count in misclassifications.items():
        print(f"Class {class_label}: {count} misclassifications")

# Load the model
model_path = "directory/ECG_Model_Lead_5.h5"
model = load_model(model_path)

# Test folder path
test_folder_path = 'directory/test_set/'

# Evaluate the test folder
evaluate_test_folder(model, test_folder_path)

(1, 96, 96, 3)
1/1 [==============================] - 0s 137ms/step
Image: A00004.png, Actual Label: 0, Predicted Label: 0, Prediction Value: 0.12873972952365875
(1, 96, 96, 3)
1/1 [==============================] - 0s 15ms/step
Image: A00225.png, Actual Label: 0, Predicted Label: 1, Prediction Value: 0.23923934996128082
(1, 96, 96, 3)
1/1 [==============================] - 0s 15ms/step
Image: A00216.png, Actual Label: 0, Predicted Label: 0, Prediction Value: 0.13327695429325104
(1, 96, 96, 3)
1/1 [==============================] - 0s 14ms/step
Image: A00267.png, Actual Label: 0, Predicted Label: 0, Prediction Value: 0.12902042269706726
(1, 96, 96, 3)
1/1 [==============================] - 0s 14ms/step
Image: A00009.png, Actual Label: 0, Predicted Label: 0, Prediction Value: 0.11362071335315704
(1, 96, 96, 3)
1/1 [==============================] - 0s 14ms/step
Image: A00231.png, Actual Label: 0, Predicted Label: 1, Prediction Value: 0.20667847990989685
(1, 96, 96, 3)
1/1 [=============